1. Import libraties and dataframes
2. Drop Extra Columns
3. Find mean, make new columns
4. perform multiple aggregations
5. Create Spender Flag
5. Create Ordering Flag

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Define path
path = r'/Users/joshd/Documents/Personal/CareerFoundry/Instacart Basket Analysis'

In [3]:
#upload ords products merge file
ords_prods_merge = pd.read_pickle(r'/Users/joshd/Documents/Personal/CareerFoundry/Instacart Basket Analysis/02 Data/Prepared Data/orders_products_merged.pkl')

In [4]:
#permantly drop a column
ords_prods_merge.drop('Unnamed: 0_x', inplace=True, axis=1)

In [5]:
#permantly drop a column
ords_prods_merge.drop('Unnamed: 0.1', inplace=True, axis=1)

In [6]:
#permantly drop a column
ords_prods_merge.drop('Unnamed: 0_y', inplace=True, axis=1)

In [7]:
ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,exists,price_range_loc,busiest_day,busiest_period_of_the_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Busiest day,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Second to Least busy,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Least busy,Average orders
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,both,both,Mid-range product,Second busiest busy,Average orders
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,2,0,both,both,Mid-range product,Regularly busy,Most orders
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.0,1,0,both,both,Mid-range product,Second to Least busy,Most orders
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.0,1,0,both,both,Mid-range product,Second busiest busy,Average orders
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.0,2,1,both,both,Mid-range product,Second busiest busy,Most orders


In [8]:
#2find mean for entire data set
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#3 The mean has changed some which is not suprising considering there were more numbers included. 

#4 label loyalty customers

In [10]:
# Create max_order column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
#label loyalty customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

#5 The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [17]:
#perform multiple aggregations
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max','median']})

prices                     
                       mean  min      max median
loyalty_flag                                    
Loyal customer    10.386336  1.0  99999.0    7.4
New customer      13.294670  1.0  99999.0    7.4
Regular customer  12.495717  1.0  99999.0    7.4

#6 The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


In [20]:
# Create average_price column
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.average)

In [21]:
#label spenders
ords_prods_merge.loc[ords_prods_merge['average_price'] <10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['average_price'] >=10, 'spending_flag'] = 'High spender'

In [22]:
ords_prods_merge['spending_flag'].value_counts(dropna=False)

Low spender     31770649
High spender      634210
Name: spending_flag, dtype: int64

#7 In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [23]:
# Create median column
ords_prods_merge['median'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [27]:
#label ordering behavior flag
ords_prods_merge.loc[ords_prods_merge['median'] >20, 'ordering_flag'] = 'Non-Frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median'] >10) & (ords_prods_merge['median'] <= 20),'ordering_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median'] <=10, 'ordering_flag'] = 'Frequent customer'

In [28]:
ords_prods_merge['ordering_flag'].value_counts(dropna=False)

Frequent customer        22796659
Regular customer          6921472
Non-Frequent customer     2686728
Name: ordering_flag, dtype: int64

In [29]:
# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))

In [30]:
ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,exists,price_range_loc,busiest_day,busiest_period_of_the_day,max_order,loyalty_flag,average_price,spending_flag,median,ordering_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest day,Average orders,5,New customer,7.930208,Low spender,6.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Second to Least busy,Most orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Least busy,Average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,both,Mid-range product,Second busiest busy,Average orders,26,Regular customer,6.935398,Low spender,11.0,Regular customer
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,both,Mid-range product,Regularly busy,Most orders,9,New customer,5.957576,Low spender,19.0,Regular customer
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,both,Mid-range product,Second to Least busy,Most orders,12,Regular customer,6.680000,Low spender,3.5,Frequent customer
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,both,Mid-range product,Second busiest busy,Average orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,both,Mid-range product,Second busiest busy,Most orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer
